In [81]:
import os 
import shutil
import random as rnd

1
# import relevant libraries
import trax
import trax.fastmath.numpy as np
from trax import layers as tl
from trax import fastmath
from trax import shapes
import pandas as pd
from sklearn.model_selection import train_test_split

# import Layer from the utils.py file
from utils import Layer, load_tweets, process_tweet
# import w1_unittest
import jax
jax.config.update('jax_platform_name', 'gpu')

import json

In [82]:
## DO NOT EDIT THIS CELL


# Import functions from the utils.py file

def train_val_split():

#############Tweeter############################
    # Load positive and negative tweets
    all_positive_tweets, all_negative_tweets = load_tweets()

    # View the total number of positive and negative tweets.
    print(f"The number of positive tweets: {len(all_positive_tweets)}")
    print(f"The number of negative tweets: {len(all_negative_tweets)}")

    # Split positive set into validation and training
    val_pos_tweeter   = all_positive_tweets[4500:] # generating validation set for positive tweets
    train_pos_tweeter  = all_positive_tweets[:4500]# generating training set for positive tweets

    # Split negative set into validation and training
    val_neg_tweeter   = all_negative_tweets[4500:] # generating validation set for negative tweets
    train_neg_tweeter  = all_negative_tweets[:4500] # generating training set for nagative tweets
    

    X_train_tweeter = train_pos_tweeter + train_neg_tweeter
    X_val_tweeter  = val_pos_tweeter + val_neg_tweeter
    y_train_tweeter = np.append(np.ones(len(train_pos_tweeter)), np.zeros(len(train_neg_tweeter)))
    y_val_tweeter  = np.append(np.ones(len(val_pos_tweeter)), np.zeros(len(val_neg_tweeter)))

############IMDB################################

    # # Load your dataset from the CSV file
    # df = pd.read_csv('imdb_dataset_review/IMDB Dataset.csv')

    # # Assuming 'sentiment' column contains 'positive' or 'negative'
    # positive_reviews_imdb = df[df['sentiment'] == 'positive']
    # negative_reviews_imdb = df[df['sentiment'] == 'negative']

    # # Split positive reviews into train and validation sets
    # train_pos_imdb, val_pos_imdb = train_test_split(positive_reviews_imdb, test_size=0.1, random_state=42)

    # # Split negative reviews into train and validation sets
    # train_neg_imdb, val_neg_imdb = train_test_split(negative_reviews_imdb, test_size=0.1, random_state=42)

    # train_dum_pos_imdb = train_pos_imdb['review'].tolist()
    # val_dum_pos_imdb = val_pos_imdb['review'].tolist()
    # train_dum_neg_imdb = train_neg_imdb['review'].tolist()
    # val_dum_neg_imdb = val_neg_imdb['review'].tolist()
    

    # # Combine train and validation sets for both positive and negative sentiments
    # train_data_imdb = pd.concat([train_pos_imdb, train_neg_imdb])
    # val_data_imdb = pd.concat([val_pos_imdb, val_neg_imdb])

    # X_train_imdb = train_data_imdb['review'].tolist()
    # X_val_imdb = val_data_imdb['review'].tolist()
    # y_train_imdb = np.where(train_data_imdb['sentiment'].values == 'positive', 1, 0)
    # y_val_imdb = np.where(val_data_imdb['sentiment'].values == 'positive', 1, 0)


############USAIR###############################
    # df = pd.read_csv('us_airlines_data/Tweets.csv')

    # # Assuming 'sentiment' column contains 'positive' or 'negative'
    # positive_reviews_usair = df[df['airline_sentiment'].isin(['positive', 'neutral'])]
    # negative_reviews_usair = df[df['airline_sentiment'] == 'negative']

    # # Split positive reviews into train and validation sets
    # train_pos_usair, val_pos_usair = train_test_split(positive_reviews_usair, test_size=0.1, random_state=42)

    # # Split negative reviews into train and validation sets
    # train_neg_usair, val_neg_usair = train_test_split(negative_reviews_usair, test_size=0.1, random_state=42)

    # train_dum_pos_usair = train_pos_usair['text'].tolist()
    # val_dum_pos_usair = val_pos_usair['text'].tolist()
    # train_dum_neg_usair = train_neg_usair['text'].tolist()
    # val_dum_neg_usair = val_neg_usair['text'].tolist()

    # # Combine train and validation sets for both positive and negative sentiments
    # train_data_usair = pd.concat([train_pos_usair, train_neg_usair])
    # val_data_usair = pd.concat([val_pos_usair, val_neg_usair])

    # X_train_usair = train_data_usair['text'].tolist()
    # X_val_usair = val_data_usair['text'].tolist()
    # y_train_usair = np.where(train_data_usair['airline_sentiment'].values == 'negative' , 0, 1)
    # y_val_usair = np.where(val_data_usair['airline_sentiment'].values == 'negative', 0, 1)

    train_pos = train_pos_tweeter  
    train_neg = train_neg_tweeter  

    # Combine all reviews and targets for training
    train_x = X_train_tweeter 
    train_y = np.concatenate([y_train_tweeter])

    # Combine all positive and negative reviews for validation
    val_pos = val_pos_tweeter 
    val_neg = val_neg_tweeter 
    # Combine all reviews and targets for validation
    val_x = X_val_tweeter 
    val_y = np.concatenate([y_val_tweeter ])


    return train_pos, train_neg, train_x, train_y, val_pos, val_neg, val_x, val_y

In [83]:
train_pos, train_neg, train_x, train_y, val_pos, val_neg, val_x, val_y = train_val_split()

print(f"length of train_x {len(train_x)}")
print(f"length of val_x {len(val_x)}")

The number of positive tweets: 5000
The number of negative tweets: 5000
length of train_x 9000
length of val_x 1000


In [84]:
# Build the vocabulary
# Unit Test Note - There is no test set here only train/val
def get_vocab(train_x, min_occurrence=2):

    # Include special tokens 
    # started with pad, end of line and unk tokens
    Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

    # Count occurrences of each word in the entire dataset
    word_counts = {}
    for tweet in train_x:
        processed_tweet = process_tweet(tweet)
        for word in processed_tweet:
            word_counts[word] = word_counts.get(word, 0) + 1

    # Add words to the vocabulary if their occurrence is greater than min_occurrence
    for word, count in word_counts.items():
        if count > min_occurrence and word not in Vocab:
            Vocab[word] = len(Vocab)

    return Vocab

# Set the minimum occurrence for a word to be included in the vocabulary
min_occurrence = 0

Vocab = get_vocab(train_x, min_occurrence)

print("Total words in vocab are", len(Vocab))
display(Vocab)

with open('Vocabulory.json', 'w') as json_file:
    json.dump(Vocab, json_file)

Vocab = None
with open('Vocabulory.json', 'r') as json_file:
    Vocab = json.load(json_file)


Total words in vocab are 9900


{'__PAD__': 0,
 '__</e>__': 1,
 '__UNK__': 2,
 'followfriday': 3,
 'for': 4,
 'be': 5,
 'top': 6,
 'engag': 7,
 'member': 8,
 'in': 9,
 'my': 10,
 'commun': 11,
 'thi': 12,
 'week': 13,
 ':)': 14,
 'hey': 15,
 'jame': 16,
 'how': 17,
 'odd': 18,
 ':/': 19,
 'pleas': 20,
 'call': 21,
 'our': 22,
 'contact': 23,
 'centr': 24,
 'on': 25,
 '02392441234': 26,
 'and': 27,
 'we': 28,
 'will': 29,
 'abl': 30,
 'to': 31,
 'assist': 32,
 'you': 33,
 'mani': 34,
 'thank': 35,
 'had': 36,
 'a': 37,
 'listen': 38,
 'last': 39,
 'night': 40,
 'as': 41,
 'bleed': 42,
 'is': 43,
 'an': 44,
 'amaz': 45,
 'track': 46,
 'when': 47,
 'are': 48,
 'scotland': 49,
 'congrat': 50,
 'yeaaah': 51,
 'yipppi': 52,
 'accnt': 53,
 'verifi': 54,
 'rqst': 55,
 'ha': 56,
 'succeed': 57,
 'got': 58,
 'blue': 59,
 'tick': 60,
 'mark': 61,
 'fb': 62,
 'profil': 63,
 '15': 64,
 'day': 65,
 'one': 66,
 'irresist': 67,
 'flipkartfashionfriday': 68,
 "don't": 69,
 'like': 70,
 'keep': 71,
 'love': 72,
 'custom': 73,
 'wait':

In [85]:
length_vocab = len(Vocab)

In [86]:
# CANDIDATE FOR TABLE TEST - If a student forgets to check for unk, there might be errors or just wrong values in the list.
# We can add those errors to check in autograder through tabled test or here student facing user test.

# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT) 
# GRADED FUNCTION: tweet_to_tensor
def tweet_to_tensor(tweet, vocab_dict, unk_token='__UNK__', verbose=False):
    '''
    Input: 
        tweet - A string containing a tweet
        vocab_dict - The words dictionary
        unk_token - The special string for unknown tokens
        verbose - Print info durign runtime
    Output:
        tensor_l - A python list with
    
    '''     
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Process the tweet into a list of words
    # where only important words are kept (stop words removed)
    word_l = process_tweet(tweet)
    
    if verbose:
        print("List of words from the processed tweet:")
        print(word_l)
    
    # Initialize the list that will contain the unique integer IDs of each word
    tensor_l = [] 
    
    # Get the unique integer ID of the __UNK__ token
    unk_ID = vocab_dict[unk_token]
    
    if verbose:
        print(f"The unique integer ID for the unk_token is {unk_ID}")
    
    # for each word in the list:
    for word in word_l:
        
        # Get the unique integer ID.
        # If the word doesn't exist in the vocab dictionary,
        # use the unique ID for __UNK__ instead.        
        word_ID = vocab_dict[word] if word in vocab_dict else unk_ID
          
        # Append the unique integer ID to the tensor list.
        tensor_l.append(word_ID)
    # tensor_l.append(1)
    ### END CODE HERE ###
    
    return tensor_l

In [87]:
print("Actual tweet is\n", val_pos[0])
print("\nTensor of tweet:\n", tweet_to_tensor(val_pos[0], Vocab))

Actual tweet is
 @delafro_ @unbearvble yeah the left one supposedly hurts more bc it's right over your heart apparently. It'll be fine! Good luck :)

Tensor of tweet:
 [197, 109, 226, 66, 9625, 934, 229, 1912, 579, 523, 591, 146, 1143, 1221, 4671, 5, 509, 288, 463, 14]


In [88]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED: Data generator
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of positive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - A numpy array specifying the importance of each example
        
    '''     

    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples    
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
    # shuffle lines if shuffle is set to True
    if shuffle:
        rnd.shuffle(pos_index_lines)
        rnd.shuffle(neg_index_lines)
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from 0 and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1


            
        ### START CODE HERE (Replace instances of 'None' with your code) ###

        # Second part: Pack n_to_take negative examples

        # Using the same batch list, start from 0 and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset,
            if neg_index >= len_data_neg :
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True 
                    break 
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
                if shuffle:
                    # Shuffle the index of the negative sample
                    rnd.shuffle(neg_index_lines)
                    
            # get the tweet as neg_index
            tweet = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = tweet_to_tensor(tweet,vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index = neg_index+1

        ### END CODE HERE ###        

        if stop:
            break;

        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:


        ### START CODE HERE (Replace instances of 'None' with your code) ###
            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = n_pad*[0]
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor + pad_l
            
            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = n_to_take*[1]
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = n_to_take*[0]
        
        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.
        example_weights = np.ones_like(targets).astype(int)

        ### END CODE HERE ###

        # note we use yield and not return
        yield inputs, targets, example_weights

In [89]:
# Set the random number generator for the shuffle procedure
rnd.seed(30) 

# Create the training data generator

def train_generator(batch_size, train_pos
                    , train_neg, vocab_dict, loop=True
                    , shuffle = False):
    return data_generator(train_pos, train_neg, batch_size, loop, vocab_dict, shuffle)

# Create the validation data generator
def val_generator(batch_size, val_pos
                    , val_neg, vocab_dict, loop=True
                    , shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, loop, vocab_dict, shuffle)

# Create the validation data generator
def test_generator(batch_size, val_pos
                    , val_neg, vocab_dict, loop=False
                    , shuffle = False):
    return data_generator(val_pos, val_neg, batch_size, loop, vocab_dict, shuffle)

# Get a batch from the train_generator and inspect.
inputs, targets, example_weights = next(train_generator(4, train_pos, train_neg, Vocab, shuffle=True))

# this will print a list of 4 tensors padded with zeros
print(f'Inputs: {inputs}')
print(f'Targets: {targets}')
print(f'Example Weights: {example_weights}')

Inputs: [[  35   72 1030  523  524  142   33  100   37   72   65   14    0    0
     0    0    0    0]
 [  35   33  468   33   77  156    0    0    0    0    0    0    0    0
     0    0    0    0]
 [ 173   69  666  101  539   86  964  109 3330  918  591  352   27  507
  4160  538  352 3887]
 [ 131  131  131  759 3887 1852  279   37 2370 6075 6665  163  474  579
    89 6564    0    0]]
Targets: [1 1 0 0]
Example Weights: [1 1 1 1]


In [90]:
# for inputs , target , example_weights in train_generator(4, train_pos, train_neg, Vocab, shuffle=True):
#     print(inputs)
import itertools

infinite_data_generator = itertools.cycle(
    train_generator(4,train_pos, train_neg, Vocab, shuffle=True))
ten_lines = [next(infinite_data_generator) for _ in range(10)]
print(len(ten_lines))
ten_lines[5]

10


(Array([[ 306, 1280,   37, 1030,    4,  183,   25,   37,   79,   43,   37,
          669,  816,   31,  508,  109,   65,   14,  487],
        [1889,  669,  669,  156,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [  10,  817, 1514,  981, 3887,   27,  173, 1122,  266,  163, 3887,
          742,    0,    0,    0,    0,    0,    0,    0],
        [6781, 6023,  368, 3887, 2043, 2043, 2043,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32),
 Array([1, 1, 0, 0], dtype=int32),
 Array([1, 1, 1, 1], dtype=int32))

In [91]:
# Test the train_generato

# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective targets)
tmp_data_gen = train_generator(batch_size = 8, train_pos=train_pos, train_neg=train_neg, vocab_dict=Vocab)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")

The inputs shape is (8, 23)
input tensor: [ 3  4  5  6  7  8  9 10 11 12 13 14  0  0  0  0  0  0  0  0  0  0  0]; target 1; example weights 1
input tensor: [15 16 17 18 19 20 21 22 23 24 25 26 27 28 29  5 30 31 32 33 14 34 35]; target 1; example weights 1
input tensor: [28 36 37 38 39 40 14 41 33 42 43 44 45 46 47 48 33  9 49  0  0  0  0]; target 1; example weights 1
input tensor: [50 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]; target 1; example weights 1
input tensor: [6307    4 3023 3887    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [ 955    9  109  308 3777  106 6308   43  131  369 4586  653 1340 2888
    9  395 1312 3887    0    0    0    0    0]; target 0; example weights 1
input tensor: [ 440 1143 6309 1069  109 1545 2549 3887    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]; target 0; example weights 1
input tensor: [1924  173 1121 152

In [92]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: SentimentAnalysisModel
def SentimentAnalysisModel(vocab_size=118675, d_model=256, n_layers=2, mode='train'):
    """Returns a sentiment analysis model.

    Args:
        vocab_size (int, optional): Size of the vocabulary. Defaults to 256.
        d_model (int, optional): Depth of embedding (n_units in the GRU cell). Defaults to 512.
        n_layers (int, optional): Number of GRU layers. Defaults to 2.
        mode (str, optional): 'train', 'eval' or 'predict', predict mode is for fast inference. Defaults to "train".

    Returns:
        trax.layers.combinators.Serial: A sentiment analysis model as a layer that maps from a tensor of tokens to a single sentiment prediction.
    """
    ### START CODE HERE ###
    model = tl.Serial( 
        tl.Embedding(vocab_size, d_model),  # Stack the embedding layer
        [tl.GRU(n_units=d_model,mode=mode) for _ in range(n_layers)],  # Stack GRU layers of d_model units keeping n_layer parameter in mind (use list comprehension syntax)
        tl.Select([0]),
        tl.Mean(axis = 1),
        tl.Dense(n_units=2),  # Adjust Dense layer for a single output unit
        tl.LogSoftmax(),  
    ) 
    return model

In [93]:
# testing your model
model = SentimentAnalysisModel(vocab_size=length_vocab)
print(model)

Serial[
  Embedding_9900_256
  GRU_256
  GRU_256
  Mean
  Dense_2
  LogSoftmax
]


In [94]:
batch_size = 16

In [95]:

print('Number of used lines from the dataset:', 2*len(train_pos))
print('Batch size (a power of 2):', int(batch_size))
steps_per_epoch = int(2*len(train_pos)/batch_size)
print('Number of steps to cover one epoch:', steps_per_epoch)

Number of used lines from the dataset: 9000
Batch size (a power of 2): 16
Number of steps to cover one epoch: 562


In [96]:
# PLEASE, DO NOT MODIFY OR DELETE THIS CELL
import tensorflow as tf
from trax.supervised import training
# from trax import

def train_model(classifier,train_pos, train_neg, val_pos, val_neg, vocab_dict, loop, batch_size , n_steps, output_dir):
    
    rnd.seed(271)

    train_task = training.TrainTask(
        labeled_data=train_generator(batch_size, train_pos
                    , train_neg, vocab_dict, loop
                    , shuffle = True),
        loss_layer=tl.CrossEntropyLoss(),
        optimizer=trax.optimizers.Adam(0.001),
        n_steps_per_checkpoint=1,
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator(batch_size, val_pos
                    , val_neg, vocab_dict, loop
                    , shuffle = True),        
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
    )

    
# return train_task, eval_task
# rnd.seed(31) # Do NOT modify this random seed. This makes the notebook easier to replicate
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###   
    # output_dir = os.path.expanduser('~/output_dir/')
    !rm -rf {output_dir}       

    training_loop = training.Loop( 
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks=[eval_task], # The evaluation task
                                output_dir=output_dir, # The output directory
                                random_seed=31 # Do not modify this random seed in order to ensure reproducibility and for grading purposes.
    ) 

    

    training_loop.run(n_steps = n_steps)
    ### END CODE HERE ###
    # print(training_loop.model.weights)

    # Load the model from SavedModel.
    # loaded_model = tf.keras.models.load_model(model_file)
        
    # Return the training_loop, since it has the model.
    return training_loop

# Train the model 1 step and keep the `trax.supervised.training.Loop` object.
output_dir = './new_model/'

try:
    shutil.rmtree(output_dir)
except OSError as e:
    pass

training_loop = train_model(SentimentAnalysisModel(), train_pos, train_neg, val_pos, val_neg, Vocab, True ,batch_size,20,output_dir)




Step      1: Total number of trainable weights: 31169282
Step      1: Ran 1 train steps in 8.29 secs
Step      1: train CrossEntropyLoss |  0.69315016
Step      1: eval  CrossEntropyLoss |  0.69314921
Step      1: eval          Accuracy |  0.50000000

Step      2: Ran 1 train steps in 7.55 secs
Step      2: train CrossEntropyLoss |  0.69349945
Step      2: eval  CrossEntropyLoss |  0.69310892
Step      2: eval          Accuracy |  0.50000000

Step      3: Ran 1 train steps in 7.33 secs
Step      3: train CrossEntropyLoss |  0.69243050
Step      3: eval  CrossEntropyLoss |  0.69183654
Step      3: eval          Accuracy |  0.50000000

Step      4: Ran 1 train steps in 7.21 secs
Step      4: train CrossEntropyLoss |  0.69615436
Step      4: eval  CrossEntropyLoss |  0.69135463
Step      4: eval          Accuracy |  0.68750000

Step      5: Ran 1 train steps in 7.25 secs
Step      5: train CrossEntropyLoss |  0.69248998
Step      5: eval  CrossEntropyLoss |  0.69192523
Step      5: eval 

In [97]:
model = SentimentAnalysisModel()
train_generator_object = train_generator(batch_size,train_pos,train_neg,Vocab)
model.init_from_file(file_name='new_model/model.pkl.gz', weights_only=True, input_signature=shapes.signature(next(train_generator_object)))

((array([[ 0.01849687,  0.01459226,  0.06136378, ..., -0.05681958,
           0.07262021, -0.02327949],
         [ 0.0585352 ,  0.00385864, -0.00041259, ...,  0.07603992,
           0.06545908, -0.076805  ],
         [-0.01325028,  0.06936208, -0.02059306, ...,  0.0189916 ,
          -0.04695515, -0.09861901],
         ...,
         [ 0.0569117 ,  0.03746707, -0.04773592, ..., -0.06541207,
          -0.07834762,  0.03926533],
         [-0.08560685,  0.04697158, -0.03526554, ..., -0.00298655,
           0.02328494,  0.07494982],
         [ 0.05376793,  0.02353536, -0.03720468, ...,  0.09931763,
           0.06046603, -0.01164777]], dtype=float32),
  (((), ((), ())),
   ((array([[ 0.00282491,  0.01119213,  0.01060081, ...,  0.00373673,
              0.00741243,  0.00113649],
            [-0.00753159, -0.00045818, -0.00037899, ...,  0.00931353,
              0.00653336, -0.00777436],
            [ 0.00036602,  0.00560274, -0.00605689, ..., -0.0032316 ,
              0.00337196,  0.0034706

In [98]:
# keras_layer = trax.AsKeras(training_loop.model)
# # training_loop.model
# print(keras_layer)

In [99]:
# weights = training_loop.model.weights
# print(weights)

In [100]:
# import tensorflow as tf

# # Create a full Keras  model using the layer from Trax.
# inputs = tf.keras.Input(shape=(None,), dtype='float32')
# hidden = keras_layer(inputs)
# # You can add other Keras layers here operating on hidden.
# outputs = hidden
# keras_model = tf.keras.Model(inputs=inputs, outputs=outputs)
# print(keras_model)

In [101]:
# import tensorflow as tf
# inputs = tf.keras.Input(shape=(None,), dtype='int32')
# hidden = keras_layer(inputs)
# outputs = hidden
# keras_model = tf.keras.Model(inputs=inputs, outputs=outputs)
# # print(keras_model)
# model_file = os.path.join(output_dir, "model_checkpoint")
# keras_model.save(model_file)

# # Load the model from SavedModel.
# loaded_model = tf.keras.models.load_model(model_file)


In [102]:
# model_file = os.path.join(output_dir, "model_checkpoint")
# model.save(model_file)

In [103]:
# training_loop.save_checkpoint('model')

In [104]:
# traing_loop_model = trax.supervised.training.Loop.load_checkpoint()

In [105]:
# help(training.Loop)

In [106]:
# Create a generator object
tmp_train_generator = train_generator(16, train_pos
                    , train_neg, Vocab, loop=True
                    , shuffle = False)



# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 23) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.


In [107]:
# feed the tweet tensors into the model to get a prediction
tmp_pred = predictions = model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


Array([[-0.55772054, -0.8498305 ],
       [-0.49686807, -0.9375993 ],
       [-0.53799534, -0.8768718 ],
       [-0.5661215 , -0.83868885],
       [-0.51582736, -0.9088347 ],
       [-0.55865717, -0.84857744],
       [-0.5376564 , -0.8773479 ],
       [-0.5395467 , -0.8746989 ],
       [-0.4795301 , -0.96515065],
       [-0.45168158, -1.0121323 ],
       [-0.46041346, -0.9970201 ],
       [-0.40933338, -1.0909203 ],
       [-0.4572921 , -1.0023806 ],
       [-0.46829343, -0.9836862 ],
       [-0.4737927 , -0.9745457 ],
       [-0.4611515 , -0.9957592 ]], dtype=float32)

In [108]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {tmp_pred[i,0]:.4f}\tPos log prob {tmp_pred[i,1]:.4f}\t is positive? {p}\t actual {tmp_targets[i]}")

Neg log prob -0.5577	Pos log prob -0.8498	 is positive? False	 actual 1
Neg log prob -0.4969	Pos log prob -0.9376	 is positive? False	 actual 1
Neg log prob -0.5380	Pos log prob -0.8769	 is positive? False	 actual 1
Neg log prob -0.5661	Pos log prob -0.8387	 is positive? False	 actual 1
Neg log prob -0.5158	Pos log prob -0.9088	 is positive? False	 actual 1
Neg log prob -0.5587	Pos log prob -0.8486	 is positive? False	 actual 1
Neg log prob -0.5377	Pos log prob -0.8773	 is positive? False	 actual 1
Neg log prob -0.5395	Pos log prob -0.8747	 is positive? False	 actual 1
Neg log prob -0.4795	Pos log prob -0.9652	 is positive? False	 actual 0
Neg log prob -0.4517	Pos log prob -1.0121	 is positive? False	 actual 0
Neg log prob -0.4604	Pos log prob -0.9970	 is positive? False	 actual 0
Neg log prob -0.4093	Pos log prob -1.0909	 is positive? False	 actual 0
Neg log prob -0.4573	Pos log prob -1.0024	 is positive? False	 actual 0
Neg log prob -0.4683	Pos log prob -0.9837	 is positive? False	 a

In [113]:
# this is used to predict on your own sentnece
def predict(sentence):
    inputs = np.array(tweet_to_tensor(sentence, vocab_dict=Vocab))
    
    # Batch size 1, add dimension for batch, to work with the model
    inputs = inputs[None, :]  
    
    # predict with the model
    preds_probs = model(inputs)
    
    print(preds_probs.shape)
    # Turn probabilities into categories
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return preds, sentiment
# try a positive sentence
sentence = "What "
tmp_pred, tmp_sentiment = predict(sentence)
print(f"The sentiment of the sentence \n***\n\"{sentence}\"\n***\nis {tmp_sentiment}.")

print()
# try a negative sentence
sentence = "Engineering is fine"
tmp_pred, tmp_sentiment = predict(sentence)
print(f"The sentiment of the sentence \n***\n\"{sentence}\"\n***\nis {tmp_sentiment}.")

(1, 2)
The sentiment of the sentence 
***
"What "
***
is negative.

(1, 2)
The sentiment of the sentence 
***
"Engineering is fine"
***
is negative.


In [110]:
# UNQ_C8 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: test_model
# UNQ_C7 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: compute_accuracy
def compute_accuracy(preds, y, y_weights):
    """
    Input: 
        preds: a tensor of shape (dim_batch, output_dim) 
        y: a tensor of shape (dim_batch,) with the true labels
        y_weights: a n.ndarray with the a weight for each example
    Output: 
        accuracy: a float between 0-1 
        weighted_num_correct (np.float32): Sum of the weighted correct predictions
        sum_weights (np.float32): Sum of the weights
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    # Create an array of booleans, 
    # True if the probability of positive sentiment is greater than
    # the probability of negative sentiment
    # else False
    is_pos = preds[:,1] > preds[:,0]

    # convert the array of booleans into an array of np.int32
    is_pos_int = is_pos.astype(np.int32)
    
    # compare the array of predictions (as int32) with the target (labels) of type int32
    correct = y == is_pos_int

    # Count the sum of the weights.
    sum_weights = np.sum(y_weights)
    
    # convert the array of correct predictions (boolean) into an arrayof np.float32
    correct_float = correct.astype(np.int32)
    
    # Multiply each prediction with its corresponding weight.
    weighted_correct_float = np.multiply(correct_float,y_weights)

    # Sum up the weighted correct predictions (of type np.float32), to go in the
    # numerator.
    weighted_num_correct = np.sum(weighted_correct_float)

    # Divide the number of weighted correct predictions by the sum of the
    # weights.
    accuracy = weighted_num_correct/sum_weights

    ### END CODE HERE ###
    return accuracy, weighted_num_correct, sum_weights

def test_model(generator, model, compute_accuracy=compute_accuracy):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0
        
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs            
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(preds=pred, y=targets, y_weights=example_weight)
                
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred += batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct/total_num_pred
    
    ### END CODE HERE ###
    return accuracy
# DO NOT EDIT THIS CELL
# testing the accuracy of your model: this takes around 20 seconds
accuracy = test_model(test_generator(16, val_pos
                    , val_neg, Vocab, loop=False
                    , shuffle = False), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.5000
